In [2]:
## Adding the .simplify_full() command may give cleaner functions to look at, but drastically increases the runtime (from 0.5 sec to more than 4 min), and, most importantly, causes completely unstable computations that ruin the precision (for unknown reasons).

prec = 200
R = RealBallField(prec)
var('x')

W = (-x/2 + sqrt(1/27 + x^2/4))^(1/3) - (x/2 + sqrt(1/27 + x^2/4))^(1/3)
W_prime = diff(W, x)
a = 1 + W/x + W_prime
b = (3*x)/2 + W

# Number of derivatives to compute
n = 7

a_derivs = [a]
b_derivs = [b]

for k in range(1, n+1):
    a_deriv_k = diff(a, x, k)
    b_deriv_k = diff(b, x, k)
    a_derivs.append(a_deriv_k)
    b_derivs.append(b_deriv_k)

# Convert symbolic functions to numerical ones evaluable on RealBallField elements
a_funcs = [fast_callable(expr, vars=[x], domain=R) for expr in a_derivs]
b_funcs = [fast_callable(expr, vars=[x], domain=R) for expr in b_derivs]

In [3]:
x0 = R(0.0001)

print("Evaluations of the derivatives of a at x =",float(x0),":")
for k in range(n+1):
    a_val = a_funcs[k](x0)
    print(f"d^{k}a/dx^{k}({x0.center():.2f}) = {a_val}")

print("\nEvaluations of the derivatives of b at x =",float(x0),":")
for k in range(n+1):
    b_val = b_funcs[k](x0)
    print(f"d^{k}b/dx^{k}({x0.center():.2f}) = {b_val}")

Evaluations of the derivatives of a at x = 0.0001 :
d^0a/dx^0(0.00) = [-0.9999999600000017999999001662669631274003983790422471895 +/- 7.56e-56]
d^1a/dx^1(0.00) = [0.000799999928000005798336938468026595776345951029044 +/- 8.55e-52]
d^2a/dx^2(0.00) = [7.9999978400002879999689929811039824949855652564 +/- 2.34e-47]
d^3a/dx^3(0.00) = [-0.043199988480001850069981468082356835946720 +/- 5.19e-43]
d^4a/dx^4(0.00) = [-431.9996544000923999834558388888214793664 +/- 3.91e-38]
d^5a/dx^5(0.00) = [6.911996304000990993444504102741236 +/- 7.71e-34]
d^6a/dx^6(0.00) = [69119.8891200495331066890978070840 +/- 7.62e-29]
d^7a/dx^7(0.00) = [-2217.59801867599812272201538 +/- 5.28e-24]

Evaluations of the derivatives of b at x = 0.0001 :
d^0b/dx^0(0.00) = [5.000000099999997239608814495811068247709449231015594972e-5 +/- 7.99e-60]
d^1b/dx^1(0.00) = [0.5000000299999985000000868752989239014661372612460925573969 +/- 2.79e-59]
d^2b/dx^2(0.00) = [0.000599999940000005068752636988434330920591240324935507947 +/- 4.35e-58]

In [38]:
def rigorous_max_partial_derivatives(f, x1, x2, y1, y2):
    var('x y')
    I_x = RIF(x1, x2)
    I_y = RIF(y1, y2)

    # dérivées partielles
    fx = diff(f, x)
    fy = diff(f, y)

    # Evaluation sur domaine 2D
    fx_eval = RIF(fx(x=I_x, y=I_y))
    fy_eval = RIF(fy(x=I_x, y=I_y))

    # Majoration de la valeur absolue
    fx_sup = max(abs(fx_eval.lower()), abs(fx_eval.upper()))
    fy_sup = max(abs(fy_eval.lower()), abs(fy_eval.upper()))

    return fx_sup, fy_sup

def riemann_integral(f_expr, x1, x2, y1, y2, n):
    
    var('x y')
    f_num = fast_callable(f_expr, vars=[x, y], domain=R)

    dx = (R(x2) - R(x1))/n
    dy = (R(y2) - R(y1))/n

    total = R(0)
    for i in range(n):
        for j in range(n):
            xi = R(x1) + i*dx
            yj = R(y1) + j*dy
            total += f_num(xi, yj) * dx * dy
        
        # Calcul du taux de complétion
        percent = int((i / n) * 100)
        sys.stdout.write(f"\rProgression : {percent}%")
        sys.stdout.flush()

    # Calcul des bornes sur les dérivées partielles
    max_fx, max_fy = max_partial_derivatives(f_expr, x1, x2, y1, y2, N_grid=100)

    # Calcul borne d'erreur théorique
    largeur = R(x2) - R(x1)
    hauteur = R(y2) - R(y1)
    error_bound = (largeur * hauteur / n) * (largeur * max_fx + hauteur * max_fy)

    return total, error_bound

# Exemple d'utilisation
var('x y')
f_expr = log(x^2+y+42)
approx_integral, error_bound = riemann_integral(f_expr, -1, 1, -1, 1, n=100)

print(f"\rApproximation de l'intégrale : {approx_integral}")
print(f"Borne d'erreur théorique (approx.) : {error_bound}")
print("L'intégrale est contenue dans l'intervalle [",float((approx_integral-error_bound).lower()) ,",",float((approx_integral+error_bound).upper()),"]")

Approximation de l'intégrale : [14.9808891904547596163520615034256312653395603419574840439 +/- 9.33e-56]
Borne d'erreur théorique (approx.) : [0.0057607433217189314750290360046457607433217189314750290360046 +/- 5.99e-62]
L'intégrale est contenue dans l'intervalle [ 14.97512844713304 , 14.98664993377648 ]
